
1. **Initial attempt**: You tried reading the Google Sheet CSV and writing directly to a UC table with `saveAsTable` in Spark Connect.

   * Result: **UnsupportedOperationException**, because Spark Connect does not support `saveAsTable` for UC tables.

2. **Attempt to write via a path**: You tried using a path like `workspace/bronze/google_sheet_daily` or `s3://…`.

   * Result: Errors like `[NO_PARENT_EXTERNAL_LOCATION_FOR_PATH]` or `Missing cloud file system scheme`, because UC requires a **registered external location** for path-based writes.

3. **Conclusion from errors**:

   * Spark Connect can read Google Sheets directly via CSV URL.
   * Persisting to UC from Spark Connect **requires either**:

     * An **external location** with proper credentials, or
     * Writing in a full Databricks notebook using `.saveAsTable`.

4. **Final summary table**:

| Goal            | Spark Connect | Managed UC                               | External Location |
| --------------- | ------------- | ---------------------------------------- | ----------------- |
| Just read sheet | Yes           | N/A                                      | No                |
| Persist to UC   | No            | Must use `.saveAsTable` in full notebook | Yes, path-based   |

---

Essentially, **reading the sheet is simple**, but **persisting into UC from Spark Connect is blocked by UC access rules**.



**Notebook purpose and execution model**

This notebook implements a daily ingestion of a Google Sheet into a Delta table. It is designed to be executed as a **Databricks Job on a fixed daily schedule**, which is the standard and production-appropriate pattern for this type of source.

**Execution pattern**

```
Databricks Job (daily)
  └── Notebook
        ├── Read Google Sheet via CSV export
        ├── Enforce / validate schema
        └── Write to Delta (overwrite or MERGE)
```

**Rationale**

* Ingestion logic is versioned with the notebook
* Execution is observable via Job runs and logs
* Refresh cadence is explicit and controlled via scheduling

**Implementation notes**

* Data is read using the Google Sheets CSV export endpoint
* The Delta table represents the authoritative daily state (unless otherwise noted)

**Scheduling**

* Configure via **Workflows → Jobs**
* Task type: Notebook
* Frequency: Daily
* Run time: off-peak (e.g. 06:00 UTC)
* Compute: small job cluster is sufficient

In [0]:
%sql
SELECT current_catalog(), current_schema();


In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS workspace.bronze;
%sql
CREATE TABLE IF NOT EXISTS workspace.bronze.google_sheet_daily
-- USING DELTA
-- LOCATION '/Volumes/bronze/google_sheet_daily';


In [0]:
%sql
CREATE TABLE workspace.bronze.google_sheet_daily
USING DELTA
LOCATION 's3://my-bucket/bronze/google_sheet_daily';


In [0]:
%sql
DESCRIBE DETAIL workspace.bronze.google_sheet_daily;


In [0]:
import requests

sheet_url = (
    "https://docs.google.com/spreadsheets/d/"
    "1-Uli6sfFq8fSTC9ViRF_-BY5l2InD1E65G9KxmjUJxE"
    "/export?format=csv&gid=0"
)

# Read CSV directly into Spark
df = (
    spark.read
    .option("header", "true")
    .option("inferSchema", "true")
    .csv(sheet_url)
)

# Overwrite the Unity Catalog table
df.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("workspace.bronze.google_sheet_daily")


In [0]:
sheet_url = (
    "https://docs.google.com/spreadsheets/d/"
    "1-Uli6sfFq8fSTC9ViRF_-BY5l2InD1E65G9KxmjUJxE"
    "/export?format=csv&gid=0"
)

df = spark.read.option("header", "true").option("inferSchema", "true").csv(sheet_url)
